In [1]:
import os
import pandas as pd

In [2]:
pd.__version__

'1.2.5'

In [3]:
# csv_path = "/home/icer/Project/openface_dir/output_vid_dir/test.csv"
csv_path = "test.csv"
df = pd.read_csv(csv_path, engine='python')

In [4]:
# 元のcsvのカラム名の先頭に空白があるため
columns_dict = {}
for column_name in df.columns:
    columns_dict[column_name] = column_name.lstrip()
columns_dict
df = df.rename(columns=columns_dict)

In [5]:
_df = df[["frame", "face_id", "timestamp", "confidence", "success", "eye_lmk_x_25", "eye_lmk_y_25", "eye_lmk_x_53", "eye_lmk_y_53"]]
_df

,frame,face_id,timestamp,confidence,success,eye_lmk_x_25,eye_lmk_y_25,eye_lmk_x_53,eye_lmk_y_53
0,1,0,0.000,0.22,0,1745.7,772.3,1774.2,775.1
1,1,1,0.000,0.03,0,584.2,798.5,615.7,793.6
2,1,2,0.000,0.93,1,41.4,719.3,83.1,714.3
3,1,3,0.000,0.98,1,2091.8,764.6,2125.7,764.0
4,2,0,0.033,0.03,0,1748.2,771.9,1775.4,776.3
...,...,...,...,...,...,...,...,...,...
2069,542,3,18.051,0.98,1,2107.9,760.7,2136.8,756.5
2070,543,0,18.085,0.88,1,1740.7,774.2,1763.6,774.8
2071,543,1,18.085,0.98,1,605.4,791.5,625.6,787.5
2072,543,2,18.085,0.98,1,49.2,694.1,87.7,689.6


In [6]:
new_df = _df.copy()
print(new_df.columns)
new_df.loc[:, ['face_coord_x']] = (new_df['eye_lmk_x_25'] + new_df['eye_lmk_x_53']) / 2
new_df.loc[:, ['face_coord_y']] = (new_df['eye_lmk_y_25'] + new_df['eye_lmk_y_53']) / 2
new_df

Index(['frame', 'face_id', 'timestamp', 'confidence', 'success',
       'eye_lmk_x_25', 'eye_lmk_y_25', 'eye_lmk_x_53', 'eye_lmk_y_53'],
      dtype='object')


,frame,face_id,timestamp,confidence,success,eye_lmk_x_25,eye_lmk_y_25,eye_lmk_x_53,eye_lmk_y_53,face_coord_x,face_coord_y
0,1,0,0.000,0.22,0,1745.7,772.3,1774.2,775.1,1759.95,773.70
1,1,1,0.000,0.03,0,584.2,798.5,615.7,793.6,599.95,796.05
2,1,2,0.000,0.93,1,41.4,719.3,83.1,714.3,62.25,716.80
3,1,3,0.000,0.98,1,2091.8,764.6,2125.7,764.0,2108.75,764.30
4,2,0,0.033,0.03,0,1748.2,771.9,1775.4,776.3,1761.80,774.10
...,...,...,...,...,...,...,...,...,...,...,...
2069,542,3,18.051,0.98,1,2107.9,760.7,2136.8,756.5,2122.35,758.60
2070,543,0,18.085,0.88,1,1740.7,774.2,1763.6,774.8,1752.15,774.50
2071,543,1,18.085,0.98,1,605.4,791.5,625.6,787.5,615.50,789.50
2072,543,2,18.085,0.98,1,49.2,694.1,87.7,689.6,68.45,691.85


In [7]:
face_id_dict = {}
for index, row in new_df[(new_df['frame'] == 1)].iterrows():
    face_id_dict[index] = (row['face_coord_x'], row['face_coord_y'])
face_id_dict

{0: (1759.95, 773.7),
 1: (599.95, 796.05),
 2: (62.25, 716.8),
 3: (2108.75, 764.3)}

In [10]:
import math
def match_face_id(face_coord, face_id_dict):
    min_face_id = None
    min_coord_diff = (float('inf'), float('inf'))
    min_dist = float('inf')
    for k, v in face_id_dict.items():
        tmp_coord_diff = (abs(v[0] - face_coord[0]), abs(v[1] - face_coord[1]))
        tmp_dist = math.sqrt(tmp_coord_diff[0] ** 2 + tmp_coord_diff[1] ** 2)
        if tmp_dist < min_dist:
            min_face_id = k
            min_coord_diff = tmp_coord_diff
            min_dist = tmp_dist
    return min_face_id, min_dist, min_coord_diff
        
counter = 0
total_dist = 0
output_df = new_df.copy()

for index, row in new_df.iterrows():
    face_coord = (row['face_coord_x'], row['face_coord_y'])
    face_id = row['face_id']
    
    new_face_id, dist, coord_diff = match_face_id(face_coord, face_id_dict)
    output_df.loc[index, 'new_face_id'] = new_face_id
    
    if face_id != new_face_id:
        print(f"[{index}] not matched!: old_id: {face_id}, new_id: {new_face_id} (dist: {dist}, diff: {coord_diff})")
        print(row)
        counter += 1
        total_dist += dist
        output_df.loc[index, 'face_id_update'] = True


print(f"\n====\nmiss match total: {counter}")
print(f"average dist: {dist / counter}")

output_df.to_csv("new_test.csv", encoding="shift_jis", index=False)

[387] not matched!: old_id: 0.0, new_id: 1 (dist: 3.731286641361108, diff: (3.1499999999999773, 2.0))
frame           105.00
face_id           0.00
timestamp         3.47
confidence        0.03
success           0.00
eye_lmk_x_25    587.20
eye_lmk_y_25    796.20
eye_lmk_x_53    619.00
eye_lmk_y_53    791.90
face_coord_x    603.10
face_coord_y    794.05
Name: 387, dtype: float64
[388] not matched!: old_id: 1.0, new_id: 0 (dist: 13.330135033074432, diff: (9.299999999999955, 9.549999999999955))
frame            105.00
face_id            1.00
timestamp          3.47
confidence         0.03
success            0.00
eye_lmk_x_25    1735.70
eye_lmk_y_25     782.20
eye_lmk_x_53    1765.60
eye_lmk_y_53     784.30
face_coord_x    1750.65
face_coord_y     783.25
Name: 388, dtype: float64
[391] not matched!: old_id: 0.0, new_id: 1 (dist: 1.320037878244345, diff: (0.7999999999999545, 1.0499999999999545))
frame           106.000
face_id           0.000
timestamp         3.504
confidence        0.030


In [11]:
pd.__version__

'1.2.5'